In [1]:
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
from pycbc import psd as psd_func
from pycbc.filter import sigma
from pesummary.gw.conversions.mass import component_masses_from_mchirp_q, q_from_eta
from simple_pe.waveforms import generate_eccentric_waveform
from simple_pe.param_est import find_peak_snr

/home/ben.patterson/.conda/envs/igwn_eccentric/lib/python3.10/site-packages/pycbc/types/array.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal as _lal


lal.MSUN_SI != Msun
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new


/home/ben.patterson/.conda/envs/igwn_eccentric/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [8]:
# Data settings
true_dict = {'ecc10sqrd': 0.2**2, 'chirp_mass': 24, 'symmetric_mass_ratio': 2/9, 'chi_eff': 0}
init_guess = {'ecc10sqrd': 0, 'chirp_mass': 23, 'symmetric_mass_ratio': 0.20, 'chi_eff': 0.1}
target_snr = 20
t_bounds = [-0.05,0.05]
f_low = 20
sample_rate = 4096
tlen = 32
print(f'Analysing data with SNR of {target_snr}.')
print(true_dict)

# Create psd
ifos = ['H1']
psds = {'H1': 'aLIGOZeroDetHighPower',
        'f_low': f_low,
        'f_high': int(sample_rate/2),
        'length': tlen,
        'delta_f': 1. / tlen
        }
psd = {}
for ifo in ifos:
    psd[ifo] = psd_func.analytical.from_string(psds[ifo], psds['length'] * psds['f_high'] + 1, psds['delta_f'],
                                                     psds['f_low'])
psd['harm'] = 1. / sum([1. / psd[ifo] for ifo in ifos])

# Generate data
true_dict['mass_ratio'] = q_from_eta(true_dict['symmetric_mass_ratio'])
true_dict['inverted_mass_ratio'] = 1/true_dict['mass_ratio']
true_dict['total_mass'] = np.sum(component_masses_from_mchirp_q(true_dict['chirp_mass'], true_dict['mass_ratio']), axis=0)
data, _ = generate_eccentric_waveform(true_dict['total_mass'], true_dict['inverted_mass_ratio'], true_dict['ecc10sqrd']**0.5,
                                               true_dict['chi_eff'], true_dict['chi_eff'], 10, sample_rate, f_ref_e=10, tlen=tlen, to_fs=False)
raw_snr = sigma(data, psd['H1'], low_frequency_cutoff=f_low, high_frequency_cutoff=psds['f_high'])
data = {'H1': data*target_snr/raw_snr}
init_guess['distance'] = raw_snr/target_snr

Analysing data with SNR of 20.
{'ecc10sqrd': 0.04000000000000001, 'chirp_mass': 24, 'symmetric_mass_ratio': 0.2222222222222222, 'chi_eff': 0}


In [9]:
# Disable pesummary warnings
_logger = logging.getLogger('PESummary')
_logger.setLevel(logging.CRITICAL + 10)

# Find peak SNR
start = time.time()
t_start, t_end = t_bounds
dx_directions = ['chirp_mass', 'symmetric_mass_ratio', 'chi_eff']
peak_dict, snr = find_peak_snr(ifos, data, psd, t_start, t_end, init_guess, dx_directions, f_low,
                               approximant="TEOBResumS", method='scipy')
base_dict = {key: peak_dict[key] for key in dx_directions}
base_dict['ecc10sqrd'] = 0
end = time.time()
print(f'Peak SNR of {snr} found in {end-start} seconds.')
print(base_dict)

XLAL Error - XLALSimIMRTEOBResumS (/home/conda/feedstock_root/build_artifacts/lalsimulation-split_1711628017380/work/lib/LALSimIMRTEOBResumS.c:610): ODE solver returned NaN radius.

XLAL Error - XLALSimIMRTEOBResumS (/home/conda/feedstock_root/build_artifacts/lalsimulation-split_1711628017380/work/lib/LALSimIMRTEOBResumS.c:610): Apparent singularity detected
XLAL Error - XLALSimInspiralChooseTDWaveform_legacy (/home/conda/feedstock_root/build_artifacts/lalsimulation-split_1711628017380/work/lib/LALSimInspiralGeneratorLegacy.c:1245): Internal function call failed: Apparent singularity detected
XLAL Error - generate_conditioned_td_waveform_from_td (/home/conda/feedstock_root/build_artifacts/lalsimulation-split_1711628017380/work/lib/LALSimInspiralGeneratorConditioning.c:136): Internal function call failed: Apparent singularity detected
XLAL Error - generate_conditioned_fd_waveform_from_td (/home/conda/feedstock_root/build_artifacts/lalsimulation-split_1711628017380/work/lib/LALSimInspira

error making waveform
error making waveform
error making waveform
error making waveform
Peak SNR of 19.18845857283769 found in 78.8346107006073 seconds.
{'chirp_mass': 23.95239855911239, 'symmetric_mass_ratio': 0.19340590535469473, 'chi_eff': 0.1025443465290291, 'ecc10sqrd': 0}
